In [1]:
# Module imports

import pandas as pd
import matplotlib.pyplot as plt

import os
import glob
import numpy as np
import brewer2mpl
import matplotlib.patches as mpatches
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
# Read input data + constants
src_file = glob.glob('./prof*.csv')
df = pd.read_csv(src_file[0],skipinitialspace=True)

In [3]:
# Process and create required DF

# Drop unrequired columns
df1 = df.drop(df.columns[[3,4,6,7,9,10,11,12,13,14,15,16,17]],axis=1)
#print df1
"""
df2 = pd.DataFrame(columns=['uid','cores','start', 'done'])
cnt=0
for row in df1.sort('uid').iterrows():
    if row[1]['executable'] == 'gmx mdrun':
        df2.loc[cnt] = [row[1]['uid'], row[1]['cores'],  row[1]['Executing'], row[1]['AgentStagingOutputPending']] 
        cnt+=1
"""

"\ndf2 = pd.DataFrame(columns=['uid','cores','start', 'done'])\ncnt=0\nfor row in df1.sort('uid').iterrows():\n    if row[1]['executable'] == 'gmx mdrun':\n        df2.loc[cnt] = [row[1]['uid'], row[1]['cores'],  row[1]['Executing'], row[1]['AgentStagingOutputPending']] \n        cnt+=1\n"

In [4]:
bmap = brewer2mpl.get_map('Set3','qualitative',len(set(df1['cores'])),reverse=True)
colors = bmap.mpl_colors
colordict = dict(zip(set(df1['cores']), colors))
df1["Color"] = df1['cores'].apply(lambda x: colordict[x])


In [19]:
# Plot Data

## Plotting variables
ids = []
proxy_ids = []
bar_min = []
bar_max = []
cnt = 1
landscape = True
color=[]
last_color=None
last_id = None
## Extract DF values to array in the same order
for row in df1.sort('uid').iterrows():
    if row[1]['executable'] == 'gmx mdrun':
    
        proxy_ids.append(cnt)
        cnt+=1
        bar_min.append(row[1]['AGENT_EXECUTING'])
        bar_max.append(row[1]['AGENT_STAGING_OUTPUT_PENDING'])
        color.append(row[1]["Color"])
        if last_color != row[1]["Color"]:
            ids.append(int(row[1]['uid'].split('.')[1]))
            last_color = row[1]["Color"]
        else:
            ids.append('')
        last_id = int(row[1]['uid'].split('.')[1])
            
        #last_row = row
        
#ids.append(int(last_row[1]['uid'].split('.')[1]))

#print len(ids)
#print bar_min
#print bar_max
        
        

fig, ax = plt.subplots()

if landscape:
    
    FONTSIZE=26
    
    ## Plot each bar in the figure individually
    for i in range(len(ids)):
        plt.bar(bottom=bar_min[i], height=bar_max[i]-bar_min[i], left=proxy_ids[i], color=color[i], align='center')

    ## Plot formatting
    ax.set_xticks(proxy_ids)
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(FONTSIZE)
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(FONTSIZE)
    ax.set_xticklabels(ids[:-1] + [last_id], fontsize=FONTSIZE)   
    #ax.grid(which='major', axis='x', linestyle='--')
    ax.set_xlabel('Task ID', fontsize=FONTSIZE)
    ax.set_ylabel('Time (seconds)', fontsize=FONTSIZE)
    ax.set_title('Execution times of varying workload', fontsize=FONTSIZE)
else:
    
    FONTSIZE=14
    
    ## Plot each bar in the figure individually
    for i in range(len(ids)):
        plt.barh(left=bar_min[i], width=bar_max[i]-bar_min[i], bottom=proxy_ids[i], color=color[i], align='center')

    ## Plot formatting
    ax.set_yticks(proxy_ids)
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(FONTSIZE)
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(FONTSIZE)
    ax.set_yticklabels(ids,fontsize=FONTSIZE)   
    #ax.grid(which='major', axis='y', linestyle='--')
    ax.set_ylabel('Unit-ids',fontsize=FONTSIZE)
    ax.set_xlabel('Time (seconds)',fontsize=FONTSIZE)
    ax.set_title('Execution times of varying workload',fontsize=FONTSIZE)


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:14: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [20]:
# Add custom legends

patch_list = []
for key,val in colordict.iteritems():
    if key != 1:
        patch_list.append(mpatches.Patch(color=val, label=str(key) + ' core(s) per task'))

    
if landscape:
    plt.legend(handles=patch_list,loc='upper left',fontsize=FONTSIZE)
else:
    plt.legend(handles=patch_list,loc='lower right',fontsize=FONTSIZE)
    
fig = plt.gcf()
fig.set_size_inches(18, 10)
fig.savefig('varying_workload_size.png', dpi=100)

[<matplotlib.patches.Patch object at 0x7f3c3eea6190>, <matplotlib.patches.Patch object at 0x7f3c386d7690>, <matplotlib.patches.Patch object at 0x7f3c3efc7650>, <matplotlib.patches.Patch object at 0x7f3c3efc71d0>, <matplotlib.patches.Patch object at 0x7f3c3f27e650>]
